# 생성한 데이터셋 업로드

In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install -qU langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("02-Evaluations")

LangSmith 추적을 시작합니다.
[프로젝트명]
02-Evaluations


In [3]:
import pandas as pd

df = pd.read_csv("data/ragas_synthetic_dataset.csv")
df.head()

,question,contexts,ground_truth,evolution_type,metadata,episode_done
0,What is the significance of AI risk management...,['1. 정책/법제 2. 기업/산업 3. 기술/연구 4. 인력/교육\n영국 과학혁신...,AI risk management is significant in the conte...,simple,"[{'source': './data/AI 브리프 2023년 12월.pdf', 'fi...",True
1,What is the significance of the LLM Hallucinat...,['1. 정책/법제 2. 기업/산업 3. 기술/연구 4. 인력/교육\n갈릴레오의 L...,The significance of the LLM Hallucination Inde...,simple,"[{'source': './data/AI 브리프 2023년 12월.pdf', 'fi...",True
2,What are the key components of AI risk managem...,"['SPRi AI Brief |\n2023-12월호\nG7, 히로시마 AI 프로세스...",The key components of AI risk management menti...,simple,"[{'source': './data/AI 브리프 2023년 12월.pdf', 'fi...",True
3,What are the key characteristics and applicati...,"['구글 딥마인드, 범용 AI 모델의 기능과 동작에 대한 분류 체계 발표\nKEY ...",The key characteristics of AGI (Artificial Gen...,simple,"[{'source': './data/AI 브리프 2023년 12월.pdf', 'fi...",True
4,What connects AI growth with data privacy and ...,"['£유튜브, 특정인을 모방한 AI 생성 콘텐츠에 대한 삭제 요청에도 대응 계획\n...",The context discusses the connection between A...,reasoning,"[{'source': './data/AI 브리프 2023년 12월.pdf', 'fi...",True


## DeepL 을 활용한 번역

- [DeepL API 키 발급](https://www.deepl.com/ko/your-account/keys)

발급 받은 키를 `.env` 파일에 `DEEPL_API_KEY` 로 저장합니다.

In [4]:
%pip install -qU deepl


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [5]:
import os
from langchain_teddynote.translate import Translator

# api키 설정
deepl_api_key = os.getenv("DEEPL_API_KEY")

# 객체 생성
translator = Translator(deepl_api_key, "EN", "KO")

# 번역 실행
translated_text = translator("hello, nice to meet you")
print(translated_text)

안녕하세요, 만나서 반가워요


In [6]:
from tqdm import tqdm

# 번역
for i, row in tqdm(df.iterrows(), total=len(df), desc="번역 진행 중"):
    df.loc[i, "question_translated"] = translator(row["question"])
    df.loc[i, "ground_truth_translated"] = translator(row["ground_truth"])

번역 진행 중: 100%|██████████| 9/9 [00:13<00:00,  1.52s/it]


In [7]:
df.head()

,question,contexts,ground_truth,evolution_type,metadata,episode_done,question_translated,ground_truth_translated
0,What is the significance of AI risk management...,['1. 정책/법제 2. 기업/산업 3. 기술/연구 4. 인력/교육\n영국 과학혁신...,AI risk management is significant in the conte...,simple,"[{'source': './data/AI 브리프 2023년 12월.pdf', 'fi...",True,AI 개발과 안전의 맥락에서 AI 위험 관리의 중요성은 무엇인가요?,AI 위험 관리는 AI 기술과 관련된 잠재적 위험을 식별하고 완화하는 것을 포함하므...
1,What is the significance of the LLM Hallucinat...,['1. 정책/법제 2. 기업/산업 3. 기술/연구 4. 인력/교육\n갈릴레오의 L...,The significance of the LLM Hallucination Inde...,simple,"[{'source': './data/AI 브리프 2023년 12월.pdf', 'fi...",True,AI 성능 평가에서 LLM 환각 지수의 의미는 무엇인가요?,AI 성능을 평가할 때 LLM 환각 지수의 중요성은 AI가 생성한 정보의 정확성을 ...
2,What are the key components of AI risk managem...,"['SPRi AI Brief |\n2023-12월호\nG7, 히로시마 AI 프로세스...",The key components of AI risk management menti...,simple,"[{'source': './data/AI 브리프 2023년 12월.pdf', 'fi...",True,G7 히로시마 프로세스 첨단 AI 시스템을 위한 국제 행동 강령에 언급된 AI 위험...,G7 히로시마 프로세스 첨단 AI 시스템 국제 행동 강령에 언급된 AI 위험 관리의...
3,What are the key characteristics and applicati...,"['구글 딥마인드, 범용 AI 모델의 기능과 동작에 대한 분류 체계 발표\nKEY ...",The key characteristics of AGI (Artificial Gen...,simple,"[{'source': './data/AI 브리프 2023년 12월.pdf', 'fi...",True,AGI(인공 일반 지능)의 주요 특징과 활용 분야는 무엇인가요?,"AGI(인공 일반 지능)의 주요 특징은 추론, 학습, 지각 등 인간과 유사한 인지 ..."
4,What connects AI growth with data privacy and ...,"['£유튜브, 특정인을 모방한 AI 생성 콘텐츠에 대한 삭제 요청에도 대응 계획\n...",The context discusses the connection between A...,reasoning,"[{'source': './data/AI 브리프 2023년 12월.pdf', 'fi...",True,AI 성장과 데이터 프라이버시 및 책임 있는 혁신의 관계는 무엇일까요?,이 문맥에서는 AI 성장과 데이터 프라이버시 사이의 연관성에 대해 논의하며 책임감 ...


번역 전

In [8]:
# question, ground_truth 열을 확인합니다.
df.loc[:, ["question", "ground_truth"]].head()

,question,ground_truth
0,What is the significance of AI risk management...,AI risk management is significant in the conte...
1,What is the significance of the LLM Hallucinat...,The significance of the LLM Hallucination Inde...
2,What are the key components of AI risk managem...,The key components of AI risk management menti...
3,What are the key characteristics and applicati...,The key characteristics of AGI (Artificial Gen...
4,What connects AI growth with data privacy and ...,The context discusses the connection between A...


번역한 결과를 확인합니다.

In [9]:
# question_translated, ground_truth_translated 열을 확인합니다.
df.loc[:, ["question_translated", "ground_truth_translated"]].head()

,question_translated,ground_truth_translated
0,AI 개발과 안전의 맥락에서 AI 위험 관리의 중요성은 무엇인가요?,AI 위험 관리는 AI 기술과 관련된 잠재적 위험을 식별하고 완화하는 것을 포함하므...
1,AI 성능 평가에서 LLM 환각 지수의 의미는 무엇인가요?,AI 성능을 평가할 때 LLM 환각 지수의 중요성은 AI가 생성한 정보의 정확성을 ...
2,G7 히로시마 프로세스 첨단 AI 시스템을 위한 국제 행동 강령에 언급된 AI 위험...,G7 히로시마 프로세스 첨단 AI 시스템 국제 행동 강령에 언급된 AI 위험 관리의...
3,AGI(인공 일반 지능)의 주요 특징과 활용 분야는 무엇인가요?,"AGI(인공 일반 지능)의 주요 특징은 추론, 학습, 지각 등 인간과 유사한 인지 ..."
4,AI 성장과 데이터 프라이버시 및 책임 있는 혁신의 관계는 무엇일까요?,이 문맥에서는 AI 성장과 데이터 프라이버시 사이의 연관성에 대해 논의하며 책임감 ...


In [10]:
# question, ground_truth 열을 삭제하고 question_translated, ground_truth_translated 열의 이름을 변경합니다.
df.drop(columns=["question", "ground_truth"], inplace=True)
df.rename(
    columns={
        "question_translated": "question",
        "ground_truth_translated": "ground_truth",
    },
    inplace=True,
)
df.head()

,contexts,evolution_type,metadata,episode_done,question,ground_truth
0,['1. 정책/법제 2. 기업/산업 3. 기술/연구 4. 인력/교육\n영국 과학혁신...,simple,"[{'source': './data/AI 브리프 2023년 12월.pdf', 'fi...",True,AI 개발과 안전의 맥락에서 AI 위험 관리의 중요성은 무엇인가요?,AI 위험 관리는 AI 기술과 관련된 잠재적 위험을 식별하고 완화하는 것을 포함하므...
1,['1. 정책/법제 2. 기업/산업 3. 기술/연구 4. 인력/교육\n갈릴레오의 L...,simple,"[{'source': './data/AI 브리프 2023년 12월.pdf', 'fi...",True,AI 성능 평가에서 LLM 환각 지수의 의미는 무엇인가요?,AI 성능을 평가할 때 LLM 환각 지수의 중요성은 AI가 생성한 정보의 정확성을 ...
2,"['SPRi AI Brief |\n2023-12월호\nG7, 히로시마 AI 프로세스...",simple,"[{'source': './data/AI 브리프 2023년 12월.pdf', 'fi...",True,G7 히로시마 프로세스 첨단 AI 시스템을 위한 국제 행동 강령에 언급된 AI 위험...,G7 히로시마 프로세스 첨단 AI 시스템 국제 행동 강령에 언급된 AI 위험 관리의...
3,"['구글 딥마인드, 범용 AI 모델의 기능과 동작에 대한 분류 체계 발표\nKEY ...",simple,"[{'source': './data/AI 브리프 2023년 12월.pdf', 'fi...",True,AGI(인공 일반 지능)의 주요 특징과 활용 분야는 무엇인가요?,"AGI(인공 일반 지능)의 주요 특징은 추론, 학습, 지각 등 인간과 유사한 인지 ..."
4,"['£유튜브, 특정인을 모방한 AI 생성 콘텐츠에 대한 삭제 요청에도 대응 계획\n...",reasoning,"[{'source': './data/AI 브리프 2023년 12월.pdf', 'fi...",True,AI 성장과 데이터 프라이버시 및 책임 있는 혁신의 관계는 무엇일까요?,이 문맥에서는 AI 성장과 데이터 프라이버시 사이의 연관성에 대해 논의하며 책임감 ...


In [11]:
# 번역한 데이터셋을 저장합니다.
df.to_csv("data/ragas_synthetic_dataset_translated.csv", index=False)

## Hugging Face Dataset 업로드

Pandas DataFrame 을 Hugging Face Dataset 으로 변환하여 업로드를 진행합니다.

In [12]:
from datasets import Dataset

# pandas DataFrame을 Hugging Face Dataset으로 변환
dataset = Dataset.from_pandas(df)

# 데이터셋 확인
print(dataset)

Dataset({
    features: ['contexts', 'evolution_type', 'metadata', 'episode_done', 'question', 'ground_truth'],
    num_rows: 9
})


In [13]:
from datasets import Dataset
import os

# pandas DataFrame을 Hugging Face Dataset으로 변환
dataset = Dataset.from_pandas(df)

# 데이터셋 이름 설정 (원하는 이름으로 변경하세요)
dataset_name = "haseom/ragas-test-dataset-kr"

# 데이터셋 업로드
dataset.push_to_hub(
    dataset_name,
    private=False,  # private=False로 설정하면 공개 데이터셋이 됩니다.
    split="ragas_v1",  # 데이터셋 split 이름 입력
    token=os.getenv("HUGGINGFACEHUB_API_TOKEN"),
)

Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

[참고] Dataset Viewer 는 표기되기까지 시간이 좀 걸릴 수 있습니다.